In [1]:
from task2.data_man.meta_data import ConllItem
from typing import AnyStr, Dict
from task2.data_man.meta_data import _assign_ner_tags, extract_spans, get_wiki_knowledge, join_tokens, get_wiki_title_knowledge, get_wiki_title_google_type, get_wiki_entities
from task2.data_man.meta_data import read_conll_item_from_file
from task2.configuration.config import logging
from task2.configuration import config

In [11]:
def analyze_coverage(conll_file: AnyStr, entity_vocab: Dict, type: AnyStr='ORG'):
    conll_items = read_conll_item_from_file(conll_file)
    hit_count = 0
    total_count = 0 + 1e-6
    for item in conll_items:
        gold_spans = extract_spans(item.labels)
        gold_entities = []
        gold_labels = []
        for k in gold_spans:
            if gold_spans[k] == 'O':
                continue
            gold_entities.append(join_tokens(item.tokens[k[0]:k[1]+1])[0])
            gold_labels.append(item.labels[k[0]][2:])
        #logging.info('{}'.format(join_tokens(tokens)[0]))    
        #logging.info('gold entities: {}'.format(gold_entities))
        #logging.info('gold labels: {}'.format(gold_labels))
        for i, entity in enumerate(gold_entities):
            if entity in entity_vocab:
                if gold_labels[i] == type:
                    hit_count += 1
                    total_count += 1
            else:
                if gold_labels[i] == type:
                    logging.info('{}'.format(join_tokens(item.tokens)[0]))    
                    logging.info('{}: {} is not in'.format(entity, gold_labels[i]))
                    total_count += 1
    return '{}%'.format(round(hit_count / total_count, 4) * 100)

In [13]:
person_entities_vocab = get_wiki_entities(config.wiki_entity_data['person'])
coverage = analyze_coverage(config.validate_file['Chinese'], person_entities_vocab, type='Cleric')
logging.info('coverage is {}'.format(coverage))

MemoryError: 

: 

In [24]:
import os
import re
import pandas as pd
GROUP_TYPE = {
    'Q4438121': 'sport organization',  
    'Q936518': 'aerospace manufacturer',
    'Q215380': 'musical group',
    'Q32178211': 'musical group',
    'Q891723': 'public corporation',
    'Q7257717': 'public corporation',
    'Q5621421': 'private corparation',
    'Q786820': 'automobile manufacturer',
    'Q18388277': 'technology company',
    'Q43229':'orginazation',
    'Q4830453':'orginazation',
}
for dir in os.walk('group_en_20230106'):

    for l in dir:
        if not isinstance(l, list):
            continue
        for file in l:
            if not file.endswith('csv'):
                continue
            csv_file = './group_en_20230106/{}'.format(file)
            print(csv_file)
            qid = csv_file.split('_')[3]
            data = pd.read_csv(csv_file, sep='\t', header=None)
            for row in data.iterrows():
                print(row)

./group_en_20230106/group_Q18388277_en_20230106_173154.csv
(0, 0    http://www.wikidata.org/entity/Q23901
1                              Codemasters
Name: 0, dtype: object)
(1, 0    http://www.wikidata.org/entity/Q24188
1                          Sega Sports R&D
Name: 1, dtype: object)
(2, 0    http://www.wikidata.org/entity/Q24228
1                           343 Industries
Name: 2, dtype: object)
(3, 0    http://www.wikidata.org/entity/Q26188
1                            Kalypso Media
Name: 3, dtype: object)
(4, 0    http://www.wikidata.org/entity/Q26222
1                        Wargaming Seattle
Name: 4, dtype: object)
(5, 0    http://www.wikidata.org/entity/Q28299
1                         Pandemic Studios
Name: 5, dtype: object)
(6, 0    http://www.wikidata.org/entity/Q45700
1                                   Konami
Name: 6, dtype: object)
(7, 0    http://www.wikidata.org/entity/Q71930
1                         Cryo Interactive
Name: 7, dtype: object)
(8, 0    http://www.wikidata.

In [4]:
 group_entities = get_wiki_entities(config.wiki_entity_data['group'])

INFO      meta_data.py line:245 get wiki entities 708114


In [5]:
group_entities

{'rozpor': ['musical group'],
 '南非米勒': ['public corporation'],
 'sabmiller': ['public corporation'],
 'sabmiller group': ['public corporation'],
 'sab miller': ['public corporation'],
 'sabmiller plc': ['public corporation'],
 'sabmiller (united kingdom)': ['public corporation'],
 'sab-miller': ['public corporation'],
 'سبمیلر': ['public corporation'],
 'south african breweries': ['orginazation', 'public corporation'],
 '英格兰橄榄球超级联赛': ['sport organization'],
 'english premiership': ['sport organization'],
 'guinness premiership': ['sport organization'],
 'aviva premiership': ['sport organization'],
 'premiership': ['sport organization'],
 'premiership rugby': ['sport organization'],
 'liga de inglaterra de rugby 15': ['sport organization'],
 'gallagher premiership rugby': ['sport organization'],
 'gallagher premiership': ['sport organization'],
 "championnat d'angleterre de rugby à xv": ['sport organization'],
 '任天堂': ['public corporation'],
 'নিনটেন্ডো': ['public corporation'],
 'নিনটে